# Ensemble
- 여러개의 분류모델을 조합해서 더 나은 성능을 내는 방법 

## RandomForest
- 의사결정트리 bagging(단일 모델 조합)에서 예측을 실행하는 방법

In [45]:
from sklearn import  datasets

import numpy as np
np.random.seed(5)

In [46]:
# 손글씨 데이터 가져오기
mnist = datasets.load_digits()
features, labels = mnist.data, mnist.target

In [47]:
len(features)

1797

In [48]:
# 의사결정 나무를 이용한 교차검증 10번 실시
from sklearn import tree
from sklearn.model_selection import cross_val_score

In [49]:

cv_scores = []
# 10번을 실행하여 교차검증 10번씩 실행 
for i in range(10):
    scores = cross_val_score(
        # 모델 적용 
        tree.DecisionTreeClassifier(),
        features,
        labels,
        cv=10,
        scoring="accuracy"
    )
    cv_scores.append(scores.mean())
print(cv_scores)

[0.8280229671011794, 0.8235630043451273, 0.8224674115456239, 0.8235692116697703, 0.8341464928615766, 0.8185692116697704, 0.8241247672253259, 0.8180136561142148, 0.8235630043451272, 0.8258038485412786]


In [50]:
# RandomForest를 이용한  교차검증 10번
from sklearn.ensemble import RandomForestClassifier
cv_scores = []

for i in range(10):
    score = cross_val_score(
        RandomForestClassifier(),
        features,
        labels,
        cv=10,
        scoring="accuracy"
    )
    cv_scores.append(scores.mean())

print(cv_scores)


[0.8258038485412786, 0.8258038485412786, 0.8258038485412786, 0.8258038485412786, 0.8258038485412786, 0.8258038485412786, 0.8258038485412786, 0.8258038485412786, 0.8258038485412786, 0.8258038485412786]


In [60]:
 # 함수 만들기 
 def cross_validation(classifier,features,labels):
    cv_scores = []
    for i in range(10):
        score = cross_val_score(
        classifier,
        features,
        labels,
        cv=10,
        scoring="accuracy"
        )
        cv_scores.append(scores.mean())
    return cv_scores

In [62]:
# 의사결정 나무 
dt_cv_scores = cross_validation(tree.DecisionTreeClassifier(),features,labels)
dt_cv_scores

[0.8258038485412786,
 0.8258038485412786,
 0.8258038485412786,
 0.8258038485412786,
 0.8258038485412786,
 0.8258038485412786,
 0.8258038485412786,
 0.8258038485412786,
 0.8258038485412786,
 0.8258038485412786]

In [63]:
# RandomForest
rf_cv_scores = cross_validation(RandomForestClassifier(),features,labels)
rf_cv_scores

[0.8258038485412786,
 0.8258038485412786,
 0.8258038485412786,
 0.8258038485412786,
 0.8258038485412786,
 0.8258038485412786,
 0.8258038485412786,
 0.8258038485412786,
 0.8258038485412786,
 0.8258038485412786]

# 랜덤포레스트와 의사결정나무의 정확도의 시각화

In [ ]:
import matplotlib.pyplot as plt

cv_list={'randrom_forest':rf_cv_scores,"decision_tree":dt_cv_scores}
df = pd.DataFrame(cv_list)
df.plot()
plt.show()

---
# 보팅(voting) 앙상블
- 단일 모델을 앙상블하여 더 나은 예측을 하는 모델 생성

In [58]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(features,labels,test_size=0.2)

### 단일 모델 정확도 측정

In [59]:
# 의사결정나무 
dtree = tree.DecisionTreeClassifier()
dtree = dtree.fit(X_train, y_train)
dtree_score = dtree.score(X_test, y_test)
print("d-tree : ", dtree_score)

d-tree :  0.8444444444444444


In [64]:
# KNN 
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier().fit(X_train, y_train)
knn_score = knn.score(X_test, y_test)
print("Knn : ",  knn_score)

Knn :  0.975


In [74]:
# SVM
from sklearn.svm import SVC
#학습
svm = SVC(probability=True).fit(X_train, y_train)
svm_score = svm.score(X_test, y_test)
print("SVM : ",svm_score)

SVM :  0.9777777777777777


### 하드 보팅
- 각각의 분류모델의 예측값들을 모아, 가장 많은 득표를  받은 예측값으로 최종 결론을 내는 방식
- 점수로 매김(다수결)
- A 모델은 ㄱ, B 모델은 ㄴ, C 모델은 ㄱ이라 할때 ㄱ을 채택 

![](../Data/Hardvoting.png)

In [72]:
from sklearn.ensemble import VotingClassifier
# 학습
voting_clf = VotingClassifier(
    # 평가방법 
    # 위에서 만든 매개변수 
    estimators =[('decision_tree',dtree),('knn',knn),('svm',svm)],
    # 정확도가 높은 걸 비중을 높여 정확도를 높인다 
    # 기울기
    weights = [0.5,1,1],
    voting = "hard"
).fit(X_train,y_train)
# 정확도
voting_clf.score(X_test,y_test)

0.9805555555555555

### 소프트 보팅
- 각각의 분류모델을 활용하여 모든 분류값들의 확률들을 더해서 가장 높은 점수를 획득한 분류값으로 최종결론을 내는 방식입니다.
- 맞는 확률
- A 모델은 {ㄱ: 0.7, ㄴ:0.3}, B 모델은 {ㄱ:0.4, ㄴ:0.6 }, C 모델으 {ㄱ:0.6,ㄴ:0.4}로 예측했을 때 
- ㄱ은 (0.7+0.4+0.6)/3 = 0.56
- ㄴ은 (0.3 +0.6 +0.4)/3 = 0.43
로 ㄱ 채택 

![](../Data/softvoting.png)

In [71]:
from sklearn.ensemble import VotingClassifier
# 학습
voting_clf = VotingClassifier(
    # 평가방법 
    # 위에서 만든 매개변수 
    estimators =[('decision_tree',dtree),('knn',knn),('svm',svm)],
    # 정확도가 높은 걸 비중을 높여 정확도를 높인다 
    # 기울기
    weights = [0.5,1,1],
    voting = "soft"
).fit(X_train,y_train)
# 정확도
voting_clf.score(X_test,y_test)

0.9805555555555555